In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

models = ['model_default',\
          'model_local_features_without_morph',\
          'model_morph_without_lemmas',\
          'model_morph_with_lemmas',\
          'model_morph_with_lemmas_and_sentences',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

        # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}

        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    training_texts.append(preprocess_text(json_to_text(file.read())))
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                nertagger.tag(text)
                text.add_layer(flatten(text['ner'], 'flat_ner'))

                for x in removed_layers:
                    text.pop_layer(x)

                text_to_json(text, file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 103.87492990493774 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 347321
Seconds required: 1.258

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 4478.370412
Trial #2 (eta = 0.200000): 6987.109825
Trial #3 (eta = 0.400000): 12761.127831
Trial #4 (eta = 0.800000): 25340.463822
Trial #5 (eta = 1.600000): 47233.711588 (worse)
Trial #6 (eta = 0.050000): 3528.484068
Tri

***** Epoch #36 *****
Loss: 970.362863
Improvement ratio: 0.377205
Feature L2-norm: 90.092581
Learning rate (eta): 0.024955
Total number of feature updates: 584748
Seconds required for this iteration: 0.457

***** Epoch #37 *****
Loss: 940.370058
Improvement ratio: 0.385845
Feature L2-norm: 90.790062
Learning rate (eta): 0.024954
Total number of feature updates: 600991
Seconds required for this iteration: 0.455

***** Epoch #38 *****
Loss: 911.497889
Improvement ratio: 0.365573
Feature L2-norm: 91.469797
Learning rate (eta): 0.024953
Total number of feature updates: 617234
Seconds required for this iteration: 0.455

***** Epoch #39 *****
Loss: 892.661497
Improvement ratio: 0.348153
Feature L2-norm: 92.134397
Learning rate (eta): 0.024951
Total number of feature updates: 633477
Seconds required for this iteration: 0.455

***** Epoch #40 *****
Loss: 865.824085
Improvement ratio: 0.341362
Feature L2-norm: 92.782184
Learning rate (eta): 0.024950
Total number of feature updates: 649720
Seco

***** Epoch #79 *****
Loss: 456.348304
Improvement ratio: 0.130998
Feature L2-norm: 110.277158
Learning rate (eta): 0.024902
Total number of feature updates: 1283197
Seconds required for this iteration: 0.454

***** Epoch #80 *****
Loss: 449.830774
Improvement ratio: 0.128516
Feature L2-norm: 110.603123
Learning rate (eta): 0.024900
Total number of feature updates: 1299440
Seconds required for this iteration: 0.457

***** Epoch #81 *****
Loss: 445.215995
Improvement ratio: 0.135039
Feature L2-norm: 110.923942
Learning rate (eta): 0.024899
Total number of feature updates: 1315683
Seconds required for this iteration: 0.458

***** Epoch #82 *****
Loss: 445.545051
Improvement ratio: 0.106830
Feature L2-norm: 111.240403
Learning rate (eta): 0.024898
Total number of feature updates: 1331926
Seconds required for this iteration: 0.482

***** Epoch #83 *****
Loss: 434.507568
Improvement ratio: 0.143469
Feature L2-norm: 111.551348
Learning rate (eta): 0.024897
Total number of feature updates: 13

***** Epoch #118 *****
Loss: 328.256324
Improvement ratio: 0.065407
Feature L2-norm: 120.604193
Learning rate (eta): 0.024853
Total number of feature updates: 1916674
Seconds required for this iteration: 0.461

***** Epoch #119 *****
Loss: 330.098587
Improvement ratio: 0.050617
Feature L2-norm: 120.821730
Learning rate (eta): 0.024852
Total number of feature updates: 1932917
Seconds required for this iteration: 0.454

***** Epoch #120 *****
Loss: 322.179260
Improvement ratio: 0.075810
Feature L2-norm: 121.036840
Learning rate (eta): 0.024851
Total number of feature updates: 1949160
Seconds required for this iteration: 0.455

***** Epoch #121 *****
Loss: 321.525729
Improvement ratio: 0.060299
Feature L2-norm: 121.248949
Learning rate (eta): 0.024850
Total number of feature updates: 1965403
Seconds required for this iteration: 0.455

***** Epoch #122 *****
Loss: 319.501280
Improvement ratio: 0.077803
Feature L2-norm: 121.459530
Learning rate (eta): 0.024848
Total number of feature update

***** Epoch #162 *****
Loss: 255.569997
Improvement ratio: 0.055061
Feature L2-norm: 128.733018
Learning rate (eta): 0.024799
Total number of feature updates: 2631366
Seconds required for this iteration: 0.455

***** Epoch #163 *****
Loss: 254.575114
Improvement ratio: 0.051870
Feature L2-norm: 128.891555
Learning rate (eta): 0.024798
Total number of feature updates: 2647609
Seconds required for this iteration: 0.454

***** Epoch #164 *****
Loss: 256.502380
Improvement ratio: 0.041851
Feature L2-norm: 129.047976
Learning rate (eta): 0.024797
Total number of feature updates: 2663852
Seconds required for this iteration: 0.457

***** Epoch #165 *****
Loss: 254.207270
Improvement ratio: 0.044543
Feature L2-norm: 129.204028
Learning rate (eta): 0.024795
Total number of feature updates: 2680095
Seconds required for this iteration: 0.459

***** Epoch #166 *****
Loss: 253.666553
Improvement ratio: 0.043229
Feature L2-norm: 129.357983
Learning rate (eta): 0.024794
Total number of feature update

...

Märgendatud fail J2rva_Tyri_V22tsa_id16384_1885a.json
Märgendatud fail Harju_Kose_Palvere_id21108_1885a.json
Märgendatud fail Tartu_R6ngu_Aakre_id4809_1889a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id13022_1876a.json
Märgendatud fail Tartu_R6ngu_Aakre_id8898_1827a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22170_1868a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id22089_1886a.json
Märgendatud fail J2rva_Anna_Purdi_id25497_1870a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17890_1885a.json
Märgendatud fail Viljandi_P6ltsamaa_Pajusi_id6831_1871a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23188_1872a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1584_1878a.json
Märgendatud fail Viljandi_P6ltsamaa_Uue-P6ltsamaa_id6410_1853a.json
Märgendatud fail L22ne_Martna_Martna_id17524_1871a.json
Märgendatud fail V6ru_Vastseliina_Misso_id24932_1886a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id4291_1862a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13301_1880a.json
Märgendatud fail L22ne_Emma

***** Epoch #11 *****
Loss: 1581.080636
Improvement ratio: 10.913315
Feature L2-norm: 76.838349
Learning rate (eta): 0.049945
Total number of feature updates: 171754
Seconds required for this iteration: 0.490

***** Epoch #12 *****
Loss: 1453.030486
Improvement ratio: 5.104338
Feature L2-norm: 78.860207
Learning rate (eta): 0.049940
Total number of feature updates: 187368
Seconds required for this iteration: 0.495

***** Epoch #13 *****
Loss: 1320.131343
Improvement ratio: 3.545848
Feature L2-norm: 80.738865
Learning rate (eta): 0.049935
Total number of feature updates: 202982
Seconds required for this iteration: 0.489

***** Epoch #14 *****
Loss: 1209.658635
Improvement ratio: 2.828507
Feature L2-norm: 82.489450
Learning rate (eta): 0.049930
Total number of feature updates: 218596
Seconds required for this iteration: 0.485

***** Epoch #15 *****
Loss: 1123.045976
Improvement ratio: 2.369050
Feature L2-norm: 84.120378
Learning rate (eta): 0.049925
Total number of feature updates: 23421

***** Epoch #52 *****
Loss: 344.679719
Improvement ratio: 0.205335
Feature L2-norm: 113.914349
Learning rate (eta): 0.049741
Total number of feature updates: 811928
Seconds required for this iteration: 0.490

***** Epoch #53 *****
Loss: 338.415870
Improvement ratio: 0.202950
Feature L2-norm: 114.370443
Learning rate (eta): 0.049736
Total number of feature updates: 827542
Seconds required for this iteration: 0.487

***** Epoch #54 *****
Loss: 341.525617
Improvement ratio: 0.214103
Feature L2-norm: 114.820222
Learning rate (eta): 0.049731
Total number of feature updates: 843156
Seconds required for this iteration: 0.486

***** Epoch #55 *****
Loss: 332.136246
Improvement ratio: 0.203483
Feature L2-norm: 115.259483
Learning rate (eta): 0.049727
Total number of feature updates: 858770
Seconds required for this iteration: 0.484

***** Epoch #56 *****
Loss: 337.717911
Improvement ratio: 0.154282
Feature L2-norm: 115.691683
Learning rate (eta): 0.049722
Total number of feature updates: 874384

***** Epoch #94 *****
Loss: 234.518824
Improvement ratio: 0.088004
Feature L2-norm: 128.146442
Learning rate (eta): 0.049534
Total number of feature updates: 1467716
Seconds required for this iteration: 0.491

***** Epoch #95 *****
Loss: 220.436114
Improvement ratio: 0.067800
Feature L2-norm: 128.398743
Learning rate (eta): 0.049529
Total number of feature updates: 1483330
Seconds required for this iteration: 0.489

***** Epoch #96 *****
Loss: 241.867015
Improvement ratio: 0.026010
Feature L2-norm: 128.654732
Learning rate (eta): 0.049525
Total number of feature updates: 1498944
Seconds required for this iteration: 0.486

***** Epoch #97 *****
Loss: 236.397242
Improvement ratio: 0.127111
Feature L2-norm: 128.906365
Learning rate (eta): 0.049520
Total number of feature updates: 1514558
Seconds required for this iteration: 0.492

***** Epoch #98 *****
Loss: 223.691606
Improvement ratio: 0.114703
Feature L2-norm: 129.154666
Learning rate (eta): 0.049515
Total number of feature updates: 15

Märgendatud fail J2rva_Peetri_V2ike-Kareda_id20031_1872a.json
Märgendatud fail Tartu_V6nnu_Ahja_id9655_1871a.json
Märgendatud fail Tartu_Kodavere_Pala_id21194_1868a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id8154_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id15090_1883a.json
Märgendatud fail Harju_Kose_Palvere_id24675_1872a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id7871_1890a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id4314_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13524_1881a.json
Märgendatud fail L22ne_Ridala_Sinalepa_id25163_1885a.json
Märgendatud fail P2rnu_Tori_Sindi_id12783_1885a.json
Märgendatud fail Tartu_N6o_Pangodi_id2812_1889a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17623_1885a.json
Märgendatud fail Harju_Kose_Palvere_id20123_1884a.json
Märgendatud fail J2rva_Tyri_Tyri-Alliku_id1866_1895a.json
Märgendatud fail P2rnu_Tori_Sindi_id7721_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16313_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17919_1885a.json
Mär

In [6]:
for model_directory in models:
    extract_results_to_txt_file(model_directory, files)

Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
